# Checking numerically that B Z_reg= C Z_psi (see definitions in the paper) for all rules with d=3,4,5

In [1]:
import numpy as np
import scipy
import sympy as sp
import itertools
from src.automata_network import *
from utils.rules import *

In [2]:
symbols=['-','+']
for rule in itertools.product(symbols, repeat=6):
    rule=list(rule)
    d=len(rule)-1
    def B(r, psi):
        return psi[0,0]**(d-1-r)*psi[1,0]**r+psi[0,1]**(d-1-r)*psi[1,1]**r
    def C(r, psi):
        sum_r_prime=0
        sum_r_double_prime=0
        for r_prime in range(d):
            binomial=scipy.special.comb(d-1,r_prime, exact=True)
            sum_r_prime+=binomial*psi[0,1]**(d-1-r_prime)*psi[1,1]**r_prime
            sum_r_double_prime+=binomial*psi[0,0]**(d-1-r_prime)*psi[1,0]**r_prime
        return sum_r_prime**r*sum_r_double_prime**(d-1-r)
    def Z_reg(psi):
        psi_0=0
        psi_1=0
        for r in range(d):
            binomial=scipy.special.comb(d-1,r, exact=True)
            psi_0+=(rule[r]=='+')*binomial*(psi[0,1]+psi[1,1])**((d-1)*r)*(psi[0,0]+psi[1,0])**((d-1)*(d-1-r))
            psi_1+=(rule[r+1]=='+')*binomial*(psi[0,1]+psi[1,1])**((d-1)*r)*(psi[0,0]+psi[1,0])**((d-1)*(d-1-r))
        return psi_0+psi_1
    def Z_psi(psi):
        psi_new=np.zeros((2,2))
        for r in range(d):
            binomial=scipy.special.comb(d-1,r, exact=True)
            psi_new[0,0]+=(rule[r]=='+')*binomial*psi[0,0]**(d-1-r)*psi[1,0]**r
            psi_new[1,0]+=(rule[r]=='+')*binomial*psi[0,1]**(d-1-r)*psi[1,1]**r
            psi_new[0,1]+=(rule[r+1]=='+')*binomial*psi[0,0]**(d-1-r)*psi[1,0]**r
            psi_new[1,1]+=(rule[r+1]=='+')*binomial*psi[0,1]**(d-1-r)*psi[1,1]**r
        return np.sum(psi_new)
    my_BP=BP(rule)
    my_BP.run()
    for r in range(d):
        if np.abs(B(r,my_BP.psi)*Z_reg(my_BP.psi)-C(r, my_BP.psi)*Z_psi(my_BP.psi))>1e-4:
            print(B(r,my_BP.psi)*Z_reg(my_BP.psi))
            print(C(r, my_BP.psi)*Z_psi(my_BP.psi))

/home/ckoller/Documents/Fixed_points_graph_CA/src/BP.py:278: RuntimeWarning: divide by zero encountered in log
  self.phi=np.log(phi_)-self.d/2*np.log(phi__)


# Checking with symbolic calculation that the relation $B(r)Z_{reg}=C(r)Z_\phi$ for d=3,4,5,6,7

Ansatz: we have the symmetries
- $\psi_{0,1}\psi_{1,0}=\psi_{0,0}\psi_{1,1}$
- $\psi_{1,0}^{d-1}=\psi_{1,1}^{d-2}\psi_{0,1}$

Then we have
$\psi_{0,1}=\frac{\psi_{1,0}^{d-1}}{\psi_{1,1}^{d-2}}$
and
$\psi_{0,0}=\frac{\psi_{0,1}\psi{1,0}}{\psi_{1,1}}=\frac{\psi_{1,0}^d}{\psi_{1,1}^{d-1}}$.

Of course the specific choice of the symmetries is arbitrary (but there are two in total, plus the normalisation).

Below we check if this works until $d=7$.

In [3]:
psi_00=sp.symbols('psi_00')
psi_01=sp.symbols('psi_01')
psi_10=sp.symbols('psi_10')
psi_11=sp.symbols('psi_11')
psi=[[psi_00,psi_01],[psi_10,psi_11]]

In [4]:
A_0=sp.symbols('A_0')
A_1=sp.symbols('A_1')
A_2=sp.symbols('A_2')
A_3=sp.symbols('A_3')
A_4=sp.symbols('A_4')
A_5=sp.symbols('A_5')
A_6=sp.symbols('A_6')
A_7=sp.symbols('A_7')

A=[A_0, A_1, A_2, A_3, A_4, A_5, A_6, A_7]
d=len(A)-1

In [5]:
def B(r, psi):
    return psi[0][0]**(d-1-r)*psi[1][0]**r+psi[0][1]**(d-1-r)*psi[1][1]**r
def C(r, psi):
    sum_r_prime=0
    sum_r_double_prime=0
    for r_prime in range(d):
        binomial=scipy.special.comb(d-1,r_prime, exact=True)
        sum_r_prime+=binomial*psi[0][1]**(d-1-r_prime)*psi[1][1]**r_prime
        sum_r_double_prime+=binomial*psi[0][0]**(d-1-r_prime)*psi[1][0]**r_prime
    return sum_r_prime**r*sum_r_double_prime**(d-1-r)
def Z_reg(psi, A):
    psi_0=0
    psi_1=0
    for r in range(d):
        binomial=scipy.special.comb(d-1,r, exact=True)
        psi_0+=A[r]*binomial*(psi[0][1]+psi[1][1])**((d-1)*r)*(psi[0][0]+psi[1][0])**((d-1)*(d-1-r))
        psi_1+=A[r+1]*binomial*(psi[0][1]+psi[1][1])**((d-1)*r)*(psi[0][0]+psi[1][0])**((d-1)*(d-1-r))
    return psi_0+psi_1
def Z_psi(psi, A):
    psi_new=[[0,0],[0,0]]
    for r in range(d):
        binomial=scipy.special.comb(d-1,r, exact=True)
        psi_new[0][0]+=A[r]*binomial*psi[0][0]**(d-1-r)*psi[1][0]**r
        psi_new[1][0]+=A[r]*binomial*psi[0][1]**(d-1-r)*psi[1][1]**r
        psi_new[0][1]+=A[r+1]*binomial*psi[0][0]**(d-1-r)*psi[1][0]**r
        psi_new[1][1]+=A[r+1]*binomial*psi[0][1]**(d-1-r)*psi[1][1]**r
    return np.sum(psi_new)

In [6]:
for r in range(d):
    f=sp.expand(B(r, psi)*Z_reg(psi, A)).subs(psi_00, psi_10**d/psi_11**(d-1)).subs(psi_01, psi_10**(d-1)/psi_11**(d-2))
    s=sp.expand(C(r, psi)*Z_psi(psi, A)).subs(psi_00, psi_10**d/psi_11**(d-1)).subs(psi_01, psi_10**(d-1)/psi_11**(d-2))
    print(sp.expand(f-s))

0
0
0
0
0
0
0
